In [175]:
import pandas as pd

In [176]:
def general_analysis(file):
    #Create Pandas Series
    f = pd.read_csv(file)
    
    #add dummy columns for thedevice and themodule (if applicable) to add up.
    f = f.join(pd.get_dummies(f['thedevice']))
    return f

In [177]:
def create_row(file, rowname):
    file[rowname] = pd.DataFrame([0 for x in range(len(file.index))])

In [184]:
def sums(file):
    return file.sum()

def shape(file):
    return file.shape

In [183]:
# Computes Pearson correlation between two columns of a file.
def correlation(file, col1, col2):
    return file[col1].corr(file[col2])

In [178]:
def wl_test_analysis(f):
    '''
    Add login_normal to check if user logged in or created account w/o social media
    Add sm to check if user created account with Facebook or Google.
    Add falselogin for when user probably clicked on social media option, but 
    did not follow through.
    This is impossible to check for non-sm option, since false login and never trying to
    create an account both result in 0.
    Do search for login with F, G, and false logins to be device specific as well.
    '''
    create_row(f, 'C')
    create_row(f, 'L')
    create_row(f, 'signin_count')
    # login is anytime someone logged in or created account. login = reg_success for co data.
    create_row(f, 'login')
    create_row(f, 'desktoplogin')
    create_row(f, 'mobilelogin')
    create_row(f, 'tabletlogin')
    create_row(f, 'login_normal')
    create_row(f, 'sm')
    create_row(f, 'falselogin')
    create_row(f, 'desktopg')
    create_row(f, 'desktopf')
    create_row(f, 'desktopnormal')
    create_row(f, 'desktop_falselogin')
    create_row(f, 'mobileg')
    create_row(f, 'mobilef')
    create_row(f, 'mobilenormal')
    create_row(f, 'mobile_falselogin')
    create_row(f, 'tabletg')
    create_row(f, 'tabletf')
    create_row(f, 'tabletnormal')
    create_row(f, 'tablet_falselogin')
    
    for i in range(len(f.index)):
        letters = f.at[i, 'login_or_create'].split(',')
        
        if 'C' in letters:
            f.at[i, 'C'] += 1
        if 'L' in letters:
            f.at[i, 'L'] += 1
        for letter in letters:
            if letter == 'L':
                f.at[i, 'signin_count'] += 1
        
        if 'C' in letters or 'L' in letters:
            f.at[i, 'login'] += 1

            if f.at[i, 'DESKTOP'] == 1:
                f.at[i, 'desktoplogin'] += 1 
            elif f.at[i, 'MOBILE'] == 1:
                f.at[i, 'mobilelogin'] += 1 
            elif f.at[i, 'TABLET'] == 1:
                f.at[i, 'tabletlogin'] += 1
                
            if f.at[i, '0'] == 1:
                f.at[i, 'login_normal'] += 1
                if f.at[i, 'DESKTOP'] == 1:
                    f.at[i, 'desktopnormal'] += 1 
                elif f.at[i, 'MOBILE'] == 1:
                    f.at[i, 'mobilenormal'] += 1 
                elif f.at[i, 'TABLET'] == 1:
                    f.at[i, 'tabletnormal'] += 1 
        
        user_check = f.at[i, 'isloggedin_r'].split(',')
        for event in user_check:
            if event == 'user':
                f.at[i, 'signin_count'] += 1
                    
            if f.at[i, 'themodule'] == 'F' or f.at[i, 'themodule'] == 'G':
                f.at[i, 'sm'] += 1 

                if f.at[i, 'themodule'] == 'F' and f.at[i, 'DESKTOP'] == 1:
                    f.at[i, 'desktopf'] += 1 
                elif f.at[i, 'themodule'] == 'G' and f.at[i, 'DESKTOP'] == 1:
                    f.at[i, 'desktopg'] += 1 
                elif f.at[i, 'themodule'] == 'F' and f.at[i, 'MOBILE'] == 1:
                    f.at[i, 'mobilef'] += 1 
                elif f.at[i, 'themodule'] == 'G' and f.at[i, 'MOBILE'] == 1:
                    f.at[i, 'mobileg'] += 1 
                elif f.at[i, 'themodule'] == 'F' and f.at[i, 'TABLET'] == 1:
                    f.at[i, 'tabletf'] += 1 
                elif f.at[i, 'themodule'] == 'G' and f.at[i, 'TABLET'] == 1:
                    f.at[i, 'tabletg'] += 1
        else:
            if f.at[i, 'themodule'] == 'F' or f.at[i, 'themodule'] == 'G':
                f.at[i, 'falselogin'] += 1
                if f.at[i, 'DESKTOP'] == 1:
                    f.at[i, 'desktop_falselogin'] += 1
                if f.at[i, 'TABLET'] == 1:
                    f.at[i, 'tablet_falselogin'] += 1
                if f.at[i, 'MOBILE'] == 1:
                    f.at[i, 'mobile_falselogin'] += 1 
    
    # Return data as a Pandas Series, inclusive of all dummy columns necessary to quickly do analysis.
    return f


In [179]:
def co_test_analysis(f):
    # Same notes and labeling as wl_test_analysis
    create_row(f, 'sm')
    create_row(f, 'login_normal')
    # Still social media specific, impossible to tell with normal login vs no login.
    create_row(f, 'falselogin')
    create_row(f, 'desktop_login')
    create_row(f, 'desktopg')
    create_row(f, 'desktopf')
    create_row(f, 'desktopnormal')
    create_row(f, 'mobile_login')
    create_row(f, 'mobileg')
    create_row(f, 'mobilef')
    create_row(f, 'mobilenormal')
    create_row(f, 'tablet_login')
    create_row(f, 'tabletg')
    create_row(f, 'tabletf')
    create_row(f, 'tabletnormal')
    
    for i in range(len(f.index)):
        if (f.at[i, 'themodule'] == 'F' or f.at[i, 'themodule'] == 'G') and f.at[i, 'reg_success'] == 1:
            f.at[i, 'sm'] += 1 
        elif (f.at[i, 'themodule'] == 'F' or f.at[i, 'themodule'] == 'G') and f.at[i, 'reg_success'] == 0:
            f.at[i, 'falselogin'] += 1 
        if (f.at[i, 'themodule'] == '0') and f.at[i, 'reg_success'] == 1:
            f.at[i, 'login_normal'] += 1 
        
        if f.at[i, 'reg_success'] == 1 and f.at[i, 'DESKTOP'] == 1:
            f.at[i, 'desktop_login'] += 1
        elif f.at[i, 'reg_success'] == 1 and f.at[i, 'MOBILE'] == 1:
            f.at[i, 'mobile_login'] += 1
        elif f.at[i, 'reg_success'] == 1 and f.at[i, 'TABLET'] == 1:
            f.at[i, 'tablet_login'] += 1
        if f.at[i, 'themodule'] == 'F' and f.at[i, 'DESKTOP'] == 1:
            f.at[i, 'desktopf'] += 1 
        elif f.at[i, 'themodule'] == 'G' and f.at[i, 'DESKTOP'] == 1:
            f.at[i, 'desktopg'] += 1 
        elif f.at[i, 'themodule'] == '0' and f.at[i, 'DESKTOP'] == 1:
            f.at[i, 'desktopnormal'] += 1
        elif f.at[i, 'themodule'] == 'F' and f.at[i, 'MOBILE'] == 1:
            f.at[i, 'mobilef'] += 1 
        elif f.at[i, 'themodule'] == 'G' and f.at[i, 'MOBILE'] == 1:
            f.at[i, 'mobileg'] += 1 
        elif f.at[i, 'themodule'] == '0' and f.at[i, 'MOBILE'] == 1:
            f.at[i, 'mobilenormal'] += 1
        elif f.at[i, 'themodule'] == 'F' and f.at[i, 'TABLET'] == 1:
            f.at[i, 'tabletf'] += 1 
        elif f.at[i, 'themodule'] == 'G' and f.at[i, 'TABLET'] == 1:
            f.at[i, 'tabletg'] += 1 
        elif f.at[i, 'themodule'] == '0' and f.at[i, 'TABLET'] == 1:
            f.at[i, 'tabletnormal'] += 1
            
    return f

In [180]:
def wl_general_analysis(f):
    '''
    Add column C to check if the user created account
    Add column L to check how many time the user logged in
    Add column signin_count to see how many events involved logging in or already being signed in
    login_total check if user was ever signed in at all, regardless of how. 
    Though some of these exact rows were created in in wl_test_analysis, I did not want to run
    the for loop an extra time by running this function on the test csv. 
    Same for co_general_analysis.
    '''
    create_row(f, 'C')
    create_row(f, 'L')
    create_row(f, 'signin_count')
    # login is anytime someone logged in or created account. login = reg_success for co data.
    create_row(f, 'login')
    create_row(f, 'desktoplogin')
    create_row(f, 'mobilelogin')
    create_row(f, 'tabletlogin')
    
    for i in range(len(f.index)):
        letters = f.at[i, 'login_or_create'].split(',')
        if 'C' in letters:
            f.at[i, 'C'] += 1
        if 'L' in letters:
            f.at[i, 'L'] += 1
        for letter in letters:
            if letter == 'L':
                f.at[i, 'signin_count'] += 1
        if 'C' in letters or 'L' in letters:
            f.at[i, 'login'] += 1

            if f.at[i, 'DESKTOP'] == 1:
                f.at[i, 'desktoplogin'] += 1 
            elif f.at[i, 'MOBILE'] == 1:
                f.at[i, 'mobilelogin'] += 1 
            elif f.at[i, 'TABLET'] == 1:
                f.at[i, 'tabletlogin'] += 1
                    
        user_check = f.at[i, 'isloggedin_r'].split(',')
        for event in user_check:
            if event == 'user':
                f.at[i, 'signin_count'] += 1
                    
    return f

In [181]:
def co_general_analysis(f):
    # Same notes and labeling as wl_general_analysis
    create_row(f, 'desktop_login')
    create_row(f, 'mobile_login')
    create_row(f, 'tablet_login')
    
    for i in range(len(f.index)):
        if f.at[i, 'reg_success'] == 1 and f.at[i, 'DESKTOP'] == 1:
            f.at[i, 'desktop_login'] += 1
        elif f.at[i, 'reg_success'] == 1 and f.at[i, 'MOBILE'] == 1:
            f.at[i, 'mobile_login'] += 1
        elif f.at[i, 'reg_success'] == 1 and f.at[i, 'TABLET'] == 1:
            f.at[i, 'tablet_login'] += 1
    
    return f

In [197]:
f = general_analysis('after_co.csv')
co_after = co_general_analysis(f)
print(sums(co_after))
# Number of users in test is first part of tuple.
print(shape(co_after))

eventcount       5816
reg_success       446
DESKTOP          2296
MOBILE            695
TABLET            352
desktop_login     361
mobile_login       45
tablet_login       40
dtype: float64
(3345, 12)


In [189]:
f = general_analysis('before_co.csv')
co_before = co_general_analysis(f)
print(sums(co_before))
# Number of users in test is first part of tuple.
print(shape(co_before))

eventcount       24412
reg_success       1921
DESKTOP           8978
MOBILE            2897
TABLET            1613
desktop_login     1520
mobile_login       200
tablet_login       201
dtype: float64
(13491, 12)


In [190]:
f = general_analysis('test_wl.csv')
wl_test = wl_test_analysis(f)
print(sums(wl_test))
# Number of users in test is first part of tuple.
print(shape(wl_test))

eventcount            15328
DESKTOP                1686
MOBILE                 1073
TABLET                  424
0                      2724
F                       329
G                       134
C                       832
L                       761
signin_count           2987
login                  1332
desktoplogin            752
mobilelogin             433
tabletlogin             147
login_normal           1012
sm                     3093
falselogin              463
desktopg                528
desktopf               1108
desktopnormal           565
desktop_falselogin      240
mobileg                 239
mobilef                 833
mobilenormal            340
mobile_falselogin       164
tabletg                 159
tabletf                 226
tabletnormal            107
tablet_falselogin        59
dtype: float64
(3187, 37)


In [186]:
f = general_analysis('test_co.csv')
co_test = co_test_analysis(f)
print(sums(co_test))
# Number of users in test is first part of tuple.
print(shape(co_test))
# Recognize that this could be bias since we don't know of the reg_success that's not SM 
# if they already had an account before test or not.

eventcount       24067
reg_success       1040
DESKTOP           9875
MOBILE            2765
TABLET            1775
0                14236
F                  124
G                   67
sm                 140
login_normal       900
falselogin          51
desktop_login      795
desktopg            47
desktopf            82
desktopnormal     9746
mobile_login       133
mobileg             12
mobilef             26
mobilenormal      2727
tablet_login       112
tabletg              8
tabletf             16
tabletnormal      1751
dtype: float64
(14427, 28)


In [194]:
f = general_analysis('after_wl.csv')
wl_after = wl_general_analysis(f)
print(sums(wl_after))
# Number of users in test is first part of tuple.
print(shape(wl_after))

eventcount      3559
DESKTOP          429
MOBILE           233
TABLET            92
C                177
L                212
signin_count     737
login            331
desktoplogin     197
mobilelogin       96
tabletlogin       38
dtype: float64
(756, 18)


In [196]:
# Not sure why exact same functions as above are providing extra data here.
f = general_analysis('before_wl.csv')
wl_before = wl_general_analysis(f)
print(sums(wl_before))
# Number of users in test is first part of tuple.
print(shape(wl_before))

_time              2016-02-28T23:32:17.809-05002016-02-28T23:26:5...
eventcount                                                     15353
firstpage          wishwishwishwishwishwishwishwishwishwishwishwi...
theugid            15765b3d1532b34b8456d6015765b3d1532b34b845740f...
thedevice          DESKTOPDESKTOPTABLETDESKTOPDESKTOPTABLETMOBILE...
isloggedin_r       user by-session,user by-session,user by-sessio...
time_hhmmss_r      23:32:17,23:32:17,23:32:1823:26:52,23:26:52,23...
login_or_create    N,N,NN,N,NN,N,NN,N,NN,N,N,C,N,N,N,NN,N,NN,N,N,...
DESKTOP                                                         1825
MOBILE                                                          1120
TABLET                                                           377
C                                                                877
L                                                                812
signin_count                                                    3057
login                             